In [ ]:
# Required imports
import csv
import logging
from datetime import datetime
import uuid

customer_ids = set()  

#Load transactions from a CSV file
def load_transactions():
    transactions = []
    try:
        with open('financial_transactions.csv', mode='r', newline='') as file:
            reader = csv.DictReader(file)
            next(reader)  # Skip header
            for row in reader:
                try:
                    date = datetime.strptime(row['date'], '%Y-%m-%d').date()
                    amount = float(row['amount'])
                    transactiontype = row['type'].lower() if 'type' in row else row[2].lower()
                    if transactiontype == 'debit':
                        amount = -amount
                    transaction = {
                        'id': row.get('id', str(uuid.uuid4())),  
                        'date': date,
                        'amount': amount,
                        'type': transactiontype,
                        'description': row['description']
                    }
                    transactions.append(transaction)
                except ValueError as e:
                    logging.error(f"Error parsing row {row}: {e}")
                    continue
        logging.info(f"Loaded {len(transactions)} transactions from file.")
        return transactions
    except Exception as e:
        logging.error(f"Failed to load transactions: {e}")
        return []
        
            
#Adding a new transaction
def add_transaction(transactions):
    logging.info("Adding a new transaction...")
    print("Adding a new transaction...")
    date_str = input("Enter transaction date (YYYY-MM-DD): ")
    amount_str = input("Enter transaction amount: ")
    transaction_type = input("Enter transaction type (credit/debit): ").strip().lower()
    description = input("Enter transaction description: ")

    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    amount = float(amount_str)

    transaction = {
        'date': date,
        'amount': amount,
        'type': transaction_type,
        'description': description
    }
    transactions.append(transaction)
    print("Transaction added successfully.")

# Function to filter transactions by year

def filter_transactions_by_year(transactions, year):
    """Return transactions that match the given year."""
    return [t for t in transactions if t['date'].year == year]

#Viewing transactions
def view_transactions(transactions):
    print("Viewing transactions...")
    if not transactions:
        print("No transactions to display.")
        return
    print("\nTransactions:")
    print(f"{'Index':<6}{'ID':<36}{'Date':<12} {'Amount':<10} {'Description'}")
    for idx, transaction in enumerate(transactions):
        date_str = transaction['date'].strftime('%Y-%m-%d')  # Format date as string
        print(f"{idx:6} {date_str:<12} {transaction['amount']:<10} {transaction['description']}")

#Updating transactions
def update_transactions(transactions):
    print("Updating transactions...")
    if not transactions:
        print("No transactions to update.")
        return transactions

    view_transactions(transactions)
    index = int(input("Enter the index of the transaction to update (0-based): "))
    
    if index < 0 or index >= len(transactions):
        print("Invalid index.")
        return transactions

    date_str = input("Enter new transaction date (YYYY-MM-DD): ")
    amount_str = input("Enter new transaction amount: ")
    type_str = input("Enter new transaction type (credit/debit): ").strip().lower()
    description = input("Enter new transaction description: ")

    transactions[index]['date'] = datetime.strptime(date_str, '%Y-%m-%d').date()
    transactions[index]['amount'] = float(amount_str)
    transactions[index]['type'] = type_str
    if type_str == 'debit':
        transactions[index]['amount'] = -transactions[index]['amount']  # Convert debit to negative amount
    transactions[index]['description'] = description
    print("Transaction updated successfully.")
    
    return transactions

# Deleting transactions
def delete_transactions(transactions):
    print("Deleting transactions...")
    if not transactions:
        print("No transactions to delete.")
        return transactions

    view_transactions(transactions)
    index = int(input("Enter the index of the transaction to delete (0-based): "))
    
    if index < 0 or index >= len(transactions):
        print("Invalid index.")
        return transactions

    del transactions[index]
    print("Transaction deleted successfully.")
    
    return transactions

# Analyzing finances
def analyze_finances(transactions):
    print("Analyzing finances...")
    if not transactions:
        print("No transactions to analyze.")
        return

    total_income = sum(t['amount'] for t in transactions if t['amount'] > 0)
    total_expense = sum(t['amount'] for t in transactions if t['amount'] < 0)
    total_transfers = sum(t['amount'] for t in transactions if t['type'] == 'transfer')
    balance = total_income + total_expense
    highest_debt = min((t['amount'] for t in transactions if t['amount'] < 0), default=0)

    print("\nFinancial Analysis:")
    print(f"Total Income: {total_income:.2f}")
    print(f"Total Expense: {total_expense:.2f}")
    print(f"Total Transfers: {total_transfers:.2f}")
    print(f"Balance: {balance:.2f}")

#Saving transactions to a CSV file
def save_transactions(transactions, filename):
    logging.info("Saving transactions to file...")
    print("Saving transactions to file...")
    try:
        with open(filename, mode='w', newline='') as file:
            fieldnames = ['id', 'date', 'amount', 'type', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for transaction in transactions:
                writer.writerow({
                    'id': transaction['id'],
                    'date': transaction['date'].strftime('%Y-%m-%d'),
                    'amount': transaction['amount'],
                    'type': transaction['type'],
                    'description': transaction['description']
                })
        print(f"Transactions saved to {filename}.")
    except Exception as e:
        print(f"An error occurred while saving transactions: {e}")

# Generating a financial report
def generate_report(transactions):
    if not transactions:
        print("No transactions to generate a report.")
        return

    total_income = sum(t['amount'] for t in transactions if t['amount'] > 0)
    total_expense = sum(t['amount'] for t in transactions if t['amount'] < 0)
    balance = total_income + total_expense

    print("\nFinancial Report:")
    print(f"Total Income: {total_income:.2f}")
    print(f"Total Expense: {total_expense:.2f}")
    print(f"Balance: {balance:.2f}")
    
    with open('financial_report.txt', 'w') as report_file:
        report_file.write("Financial Report:\n")
        report_file.write(f"Total Income: {total_income:.2f}\n")
        report_file.write(f"Total Expense: {total_expense:.2f}\n")
        report_file.write(f"Balance: {balance:.2f}\n")
        print("Report generated and saved to financial_report.txt.")
    print("Report generation complete.")

#Main function to run the application
def main():
    logging.basicConfig(level=logging.DEBUG,)
    logging.debug("Starting the Personal Finance Analyzer application.")
    transactions = []
    while True:
        print("\nPersonal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        if choice == '1':
            logging.debug("Loading transactions from file financial_transactions.csv")
            print("Loading transactions...")
            filename = 'financial_transactions.csv'
            transactions = load_transactions()
            if transactions:
                print(f"Loaded {len(transactions)} transactions.")
            else:
                print("No transactions loaded or file not found.")
        elif choice == '2':
            logging.debug("Adding a new transaction...")
            print("Adding a new transaction...")
            if not transactions:
                print("No transactions loaded. Please load transactions first.")
                continue
            try:
                print("Adding a new transaction...")
                add_transaction(transactions)
            except ValueError as e:
                print(f"Error adding transaction: {e}")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
        elif choice == '3':
            logging.debug("Viewing transactions...")
            print("Viewing transactions...")
            view_transactions(transactions)
            year = int(input("Enter year to filter by (e.g., 2024): "))
            filtered = filter_transactions_by_year(transactions, year)
            view_transactions(filtered)         
        elif choice == '4':
            logging.debug("Updating transactions...")
            print("Updating transactions...")
            transactions= update_transactions(transactions)
        elif choice == '5':
            logging.debug("Deleting transactions...")
            print("Deleting transactions...")
            transactions = delete_transactions(transactions)
        elif choice == '6':
            logging.debug("Analyzing finances...")
            print("Analyzing finances...")
            analyze_finances(transactions)
        elif choice == '7':
            logging.debug("Saving transactions to file financial_transactions.csv")
            print("Saving transactions to file...")
            save_transactions(transactions, 'financial_transactions.csv')
        elif choice == '8':
            logging.debug("Generating financial report...")  
            generate_report(transactions)
        elif choice == '9':
            logging.debug("Exiting the program...")
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        print("An unexpected error occurred. Please check the logs for more details.")


